In [1]:
import socket
import pickle
import tkinter as tk
from tkinter import ttk, messagebox
from ttkthemes import ThemedStyle
from PIL import Image, ImageTk
import sympy
import random

def calculate_prime_modulus(secret):
    # Find the first prime number after the given 'secret'
    prime_modulus = sympy.nextprime(secret + 1)
    return prime_modulus

def CreatePolynomial(coefficients, x, prime_modulus):
    polynomial = 0
    for i, coef in enumerate(coefficients):
        polynomial = (polynomial + coef * (x ** i)) % prime_modulus
    return polynomial

def CreateShares(secret, threshold, prime_modulus, NumberOfShares):
    coefficients = [secret] + [random.randint(1, prime_modulus - 1) for _ in range(threshold - 1)]

    shares = []
    for i in range(1, NumberOfShares + 1):
        x = random.randint(1, prime_modulus - 1)
        y = CreatePolynomial(coefficients, x, prime_modulus) % prime_modulus
        shares.append((x, y))

    return shares

def lagrange_interpolation(shares, prime_modulus):
    secret_reconstructed = 0
    for i, yi in shares:
        numerator = denominator = 1
        for j, yj in shares:
            if i != j:
                numerator = (numerator * -j) % prime_modulus
                denominator = (denominator * (i - j)) % prime_modulus
        lagrange_term = (yi * numerator * pow(denominator, -1, prime_modulus)) % prime_modulus
        secret_reconstructed = (secret_reconstructed + lagrange_term) % prime_modulus

    return secret_reconstructed

def show_share_selection_window(total_shares):
    share_selection_window = tk.Toplevel(root)
    share_selection_window.title("Select Shares")

    # Checkboxes for selecting shares in two rows
    share_var = {}
    num_rows = (total_shares + 1) // 2
    for i in range(1, total_shares + 1):
        row = (i - 1) // num_rows
        share_var[i] = tk.IntVar()
        share_checkbox = ttk.Checkbutton(share_selection_window, text=f"Use Share {i}", variable=share_var[i])
        share_checkbox.grid(row=row, column=i % num_rows, padx=10, pady=5)

    login_button = ttk.Button(share_selection_window, text="Login", command=lambda: login(share_var))
    login_button.grid(row=num_rows, column=0, columnspan=num_rows, padx=10, pady=10)

def connect_to_server():
    # Specify the host and port of the server
    host = '192.168.19.195'  # Change this to the actual IP address of the server
    port = 8080  # Change this to the actual port number used by the server

    # Create a socket object and connect to the server
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        client_socket.connect((host, port))
    except ConnectionError as e:
        messagebox.showerror("Connection Error", f"Failed to connect to the server: {e}")
        return None

    return client_socket

def send_data_to_server(data):
    # Connect to the server
    client_socket = connect_to_server()

    if client_socket is not None:
        try:
            # Pickle and send the data to the server
            data_pickled = pickle.dumps(data)
            client_socket.sendall(data_pickled)
            return True
        except Exception as e:
            messagebox.showerror("Error", f"Failed to send data to the server: {e}")
        finally:
            # Close the connection
            client_socket.close()

    return False

def login(share_var):
    # Retrieve values from the input fields
    original_secret = int(secret_var.get())
    threshold = int(threshold_var.get())
    total_shares = int(total_shares_var.get())

    # Generate prime modulus based on the original secret
    prime_modulus = calculate_prime_modulus(original_secret)

    if prime_modulus <= threshold:
        messagebox.showerror("Error", "Prime modulus must be greater than the threshold.")
        return

    # Retrieve selected shares for reconstruction
    selected_shares = []
    for i in range(1, total_shares + 1):
        if share_var[i].get():
            selected_shares.append(i)

    # Check if the number of selected shares meets the threshold
    if len(selected_shares) < threshold:
        messagebox.showerror("Error", "The number of shares selected does not meet the threshold.")
        return

    # Step 2: Create shares
    shares = CreateShares(original_secret, threshold, prime_modulus, total_shares)

    # Step 4: Reconstruct the secret using selected shares
    reconstructed_secret = lagrange_interpolation([shares[i - 1] for i in selected_shares], prime_modulus)

    # Check if the reconstructed secret matches the original secret
    if reconstructed_secret == original_secret:
        # Create a dictionary to send data to the server
        data_to_send = {
            'action': 'login',
            'reconstructed_secret': reconstructed_secret
        }

        # Send data to the server
        if send_data_to_server(data_to_send):
            messagebox.showinfo("Login Successful", "Password is correct. Login successful!")
    else:
        messagebox.showerror("Login Failed", "Password is incorrect. Login failed.")

def on_total_shares_entry():
    total_shares = int(total_shares_var.get())
    show_share_selection_window(total_shares)

root = tk.Tk()
root.title("Shamir's Secret Sharing Login System")

style = ThemedStyle(root)
style.set_theme("plastik")

image = Image.open("lock.png") 
image = image.resize((150, 150))
photo = ImageTk.PhotoImage(image)

image_label = ttk.Label(root, image=photo)
image_label.pack()

frame = ttk.Frame(root)
frame.pack(pady=20)

secret_label = ttk.Label(frame, text="Enter Secret:")
secret_label.grid(row=0, column=0, padx=10, pady=10, sticky='w')
secret_var = tk.StringVar()
secret_entry = ttk.Entry(frame, textvariable=secret_var)
secret_entry.grid(row=0, column=1, padx=10, pady=10)

threshold_label = ttk.Label(frame, text="Enter Threshold:")
threshold_label.grid(row=1, column=0, padx=10, pady=10, sticky='w')
threshold_var = tk.StringVar()
threshold_entry = ttk.Entry(frame, textvariable=threshold_var)
threshold_entry.grid(row=1, column=1, padx=10, pady=10)

total_shares_label = ttk.Label(frame, text="Enter Total Shares:")
total_shares_label.grid(row=2, column=0, padx=10, pady=10, sticky='w')
total_shares_var = tk.StringVar()
total_shares_entry = ttk.Entry(frame, textvariable=total_shares_var)
total_shares_entry.grid(row=2, column=1, padx=10, pady=10)
total_shares_entry.bind('<Return>', lambda event=None: on_total_shares_entry())

next_button = ttk.Button(frame, text="Next", command=on_total_shares_entry)
next_button.grid(row=3, column=0, columnspan=2, padx=10, pady=10)

root.mainloop()
